# Problem Statement 4: Total Points Prediction
## Predict Team Total Points in Season

**Author:** ScoreSight ML Team  
**Date:** 2025-11-12  
**Problem Type:** Regression (Points Tally)

### Dataset
- **File:** `data/data_engineered_league_points.csv`
- **Task:** Predict team's total season points
- **Features:** Team statistics (22+ engineered features)
- **Target:** target_total_points

## 1. Setup

In [ ]:
import pandas as pd
import numpy as np
import json
import joblib
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

try:
    import xgboost as xgb
    XGB_AVAILABLE = True
except ImportError:
    XGB_AVAILABLE = False

try:
    import lightgbm as lgb
    LGB_AVAILABLE = True
except ImportError:
    LGB_AVAILABLE = False

# GitHub configuration
GITHUB_REPO = 'https://raw.githubusercontent.com/springboardmentor345a-create/Projects_2/Prathamesh_Fuke'
DATA_URL_BASE = f'{GITHUB_REPO}/data/engineered'

models_dir = Path('models')
models_dir.mkdir(exist_ok=True, parents=True)

print("[OK] All libraries imported")
print(f"XGBoost: {XGB_AVAILABLE} | LightGBM: {LGB_AVAILABLE}")

## 2. Load Data

In [ ]:
# Load data (same file as PS1, different target)
# Use GitHub URL if on Colab, local path otherwise
import os
if os.path.exists('../data/corrected/league_winner_with_top4.csv'):
    data_path = '../data/corrected/league_winner_with_top4.csv'
    print(f"[LOAD] Loading from local file: {data_path}")
else:
    data_path = 'https://raw.githubusercontent.com/springboardmentor345a-create/Projects_2/Prathamesh_Fuke/data/corrected/league_winner_with_top4.csv'
    print(f"[LOAD] Loading from GitHub: {data_path}")

df = pd.read_csv(data_path)
df.columns = df.columns.str.lower().str.strip()

print(f"[OK] Shape: {df.shape}")
print(f"\n📊 Target (target_total_points) Statistics:")
print(f"  Mean: {df['target_total_points'].mean():.2f} points")
print(f"  Std: {df['target_total_points'].std():.2f}")
print(f"  Min: {df['target_total_points'].min()}, Max: {df['target_total_points'].max()}")
print(f"  Median: {df['target_total_points'].median():.1f}")

# Prepare features - exclude ALL targets and identifiers
exclude_cols = ['target_total_points', 'target_league_position', 'target_champion', 'top4',
                'unnamed:_0', 'season_encoded', 'team_encoded', 'team', 'season']
feature_cols = [col for col in df.columns 
               if col not in exclude_cols and df[col].dtype in ['float64', 'int64']]

X = df[feature_cols].copy()
y = df['target_total_points'].copy()

# Handle missing values
if X.isnull().sum().sum() > 0:
    print(f"\n⚠️  Found {X.isnull().sum().sum()} missing values - filling with column means")
    X = X.fillna(X.mean())

print(f"\n[OK] Features: {feature_cols}")
print(f"[OK] Number of features: {len(feature_cols)}")
print(f"[OK] Samples: {len(X)}")

# Train/test split (80/20)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\n[SPLIT] Train: {len(X_train)}, Test: {len(X_test)}")
print(f"[SPLIT] Train target stats: Mean={y_train.mean():.2f}, Std={y_train.std():.2f}")
print(f"[SPLIT] Test target stats: Mean={y_test.mean():.2f}, Std={y_test.std():.2f}")

## 3. Train Models

In [ ]:
# ============================================================================
# TRAIN/TEST SPLIT
# ============================================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"[SPLIT] Train: {len(X_train)}, Test: {len(X_test)}")
print(f"[SPLIT] Train target - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}")
print(f"[SPLIT] Test target  - Mean: {y_test.mean():.2f}, Std: {y_test.std():.2f}")

# ============================================================================
# HYPERPARAMETER SEARCH SPACE - Total Points Regression (Small Dataset)
# ============================================================================

param_grids = {
    'Ridge': {
        'model__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
    },
    'Lasso': {
        'model__alpha': [0.001, 0.01, 0.1, 1, 10]
    },
    'RandomForest': {
        'model__n_estimators': [50, 100, 150],
        'model__max_depth': [5, 10, 15, None],
        'model__min_samples_split': [2, 5],
        'model__min_samples_leaf': [1, 2]
    },
    'GradientBoosting': {
        'model__n_estimators': [50, 100, 150],
        'model__learning_rate': [0.01, 0.05, 0.1],
        'model__max_depth': [3, 5, 7],
        'model__subsample': [0.8, 0.9]
    }
}

if XGB_AVAILABLE:
    param_grids['XGBoost'] = {
        'model__n_estimators': [50, 100, 150],
        'model__learning_rate': [0.01, 0.05, 0.1],
        'model__max_depth': [3, 5, 7],
        'model__subsample': [0.8, 0.9]
    }

if LGB_AVAILABLE:
    param_grids['LightGBM'] = {
        'model__n_estimators': [50, 100, 150],
        'model__learning_rate': [0.01, 0.05, 0.1],
        'model__num_leaves': [20, 30, 40]
    }

# ============================================================================
# TRAINING WITH HYPERPARAMETER TUNING
# ============================================================================

def create_pipeline(model, use_scaling=True):
    steps = [('imputer', SimpleImputer(strategy='mean'))]
    if use_scaling:
        steps.append(('scaler', StandardScaler()))
    steps.append(('model', model))
    return Pipeline(steps)

print("\n" + "="*80)
print("HYPERPARAMETER TUNING - RandomizedSearchCV (Regression)")
print("="*80)

results = {}
trained_models = {}
best_models = {}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

models_to_train = {
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42, max_iter=10000),
    'RandomForest': RandomForestRegressor(random_state=42, n_jobs=-1),
    'GradientBoosting': GradientBoostingRegressor(random_state=42)
}

if XGB_AVAILABLE:
    models_to_train['XGBoost'] = xgb.XGBRegressor(random_state=42, n_jobs=-1)

if LGB_AVAILABLE:
    models_to_train['LightGBM'] = lgb.LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1)

for model_name, model in models_to_train.items():
    print(f"\n{'-'*80}")
    print(f"[{model_name}] Hyperparameter Tuning...")
    print(f"{'-'*80}")
    
    pipeline = create_pipeline(model)
    param_grid = param_grids[model_name]
    
    search = RandomizedSearchCV(
        pipeline,
        param_grid,
        n_iter=20,
        cv=cv,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    
    search.fit(X_train, y_train)
    
    best_models[model_name] = search.best_estimator_
    print(f"\n[BEST] Params: {search.best_params_}")
    print(f"[CV] MAE: {-search.best_score_:.4f}")
    
    # Test evaluation
    y_pred = best_models[model_name].predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    results[model_name] = {
        'best_params': search.best_params_,
        'cv_mae': float(-search.best_score_),
        'test_mae': float(mae),
        'test_rmse': float(rmse),
        'test_r2': float(r2)
    }
    
    print(f"\n[TEST] MAE:  {mae:.4f}")
    print(f"[TEST] RMSE: {rmse:.4f}")
    print(f"[TEST] R²:   {r2:.4f}")
    
    # Residuals analysis
    residuals = y_test - y_pred
    print(f"[TEST] Residuals - Mean: {residuals.mean():.4f}, Std: {residuals.std():.4f}")
    
    trained_models[model_name] = best_models[model_name]

print("\n" + "="*80)
print("TRAINING COMPLETE")
print("="*80)

## 4. Save Models

In [ ]:
# ============================================================================
# IDENTIFY BEST MODEL
# ============================================================================

best_model_name = min(results, key=lambda x: results[x]['test_mae'])
best_metrics = results[best_model_name]

print(f"\n[WINNER] Best Model: {best_model_name}")
print(f"[WINNER] Test MAE: {best_metrics['test_mae']:.4f}")
print(f"[WINNER] Test RMSE: {best_metrics['test_rmse']:.4f}")
print(f"[WINNER] Test R²: {best_metrics['test_r2']:.4f}")

# ============================================================================
# SAVE ONLY THE BEST MODEL
# ============================================================================

print(f"\n{'='*80}")
print("SAVING BEST MODEL")
print(f"{'='*80}")

# Save only the best model
best_model_path = models_dir / 'ps4_total_points_best_model.joblib'
joblib.dump(trained_models[best_model_name], best_model_path)
print(f"[SAVE] Best Model ({best_model_name}) -> {best_model_path}")

# ============================================================================
# SAVE TRAINING SUMMARY
# ============================================================================

summary = {
    'problem_statement': 'PS4: Total Points Tally Prediction',
    'task_type': 'Regression',
    'best_model': best_model_name,
    'best_test_mae': best_metrics['test_mae'],
    'best_test_rmse': best_metrics['test_rmse'],
    'best_test_r2': best_metrics['test_r2'],
    'timestamp': datetime.now().isoformat(),
    'data': {
        'path': data_path,
        'shape': list(df.shape),
        'train_size': len(X_train),
        'test_size': len(X_test),
        'n_features': len(feature_cols)
    },
    'features': feature_cols,
    'cv_strategy': '5-Fold KFold',
    'tuning_method': 'RandomizedSearchCV (20 iterations)',
    'scoring_metric': 'neg_mean_absolute_error',
    'best_params': best_metrics['best_params']
}

summary_path = models_dir / 'ps4_total_points_metadata.json'
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2, default=str)

print(f"[SAVE] Metadata -> {summary_path}")

print("\n" + "="*80)
print("✅ PS4: TOTAL POINTS PREDICTION - COMPLETE!")
print("="*80)